## SCRAPPING_GITHUB_TOPIC_RESPIRATORIES

#### Use the request library to download web pages

In [ ]:
!pip install requests --upgrade --quiet

In [ ]:
import requests

In [ ]:
topics_url = "https://github.com/topics"

In [ ]:
response = requests.get(topics_url)

In [ ]:
response.status_code

In [ ]:
page_contents = response.text

In [ ]:
page_contents[:1000]

In [ ]:
with open('webpage.html', 'w') as f:
    f.write(page_contents)

#### Use Beautiful Soup to parse and extract information

In [ ]:
!pip install beautifulsoup4 --upgrade --quiet

In [ ]:
from bs4 import BeautifulSoup  

In [ ]:
doc = BeautifulSoup(page_contents, 'html.parser')

In [ ]:
topic_title_tags = doc.find_all('p')

In [ ]:
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
topic_title_tags = doc.find_all('p', {'class': selection_class})

In [ ]:
len(topic_title_tags)

In [ ]:
topic_title_tags[:5]

In [ ]:
topic_desc_tags = doc.find_all('p')

In [ ]:
selection_class = 'f5 color-fg-muted mb-0 mt-1'
topic_desc_tags = doc.find_all('p', {'class': selection_class})

In [ ]:
topic_desc_tags[:5]

In [ ]:
topic_title_tag0 = topic_title_tags[0]

In [ ]:
div_tag = topic_title_tag0.parent

In [ ]:
topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-grow-0'})

In [ ]:
topic0_url = 'https://github.com' + topic_link_tags[0] ['href']
print(topic0_url)

In [ ]:
topic_titles = []

for tag in topic_title_tags:
    topic_titles.append(tag.text)
print(topic_titles)    

In [ ]:
topic_descriptions = []

for tag in topic_desc_tags:
    topic_descriptions.append(tag.text.strip())
    
print(topic_descriptions[:5])    

In [ ]:
topic_urls = []
base_url = 'https://github.com'

for tag in topic_link_tags:
    topic_urls.append(base_url + tag['href'])

topic_urls    

In [ ]:
!pip install pandas --quiet

In [ ]:
import pandas as pd

In [ ]:
topics_dict = {
    'Title' : topic_titles,
    'Description' : topic_descriptions,
    'URL' : topic_urls
    
}

In [ ]:
topic_df = pd.DataFrame(topics_dict)

In [ ]:
topic_df

#### Create CSV file(s) with the extracted information

In [ ]:
topic_df.to_csv('topics.csv', index= None)

#### Getting information out of a topic page

In [ ]:
topic_page_url = topic_urls[0]

In [ ]:
topic_page_url

In [ ]:
response = requests.get(topic_page_url)

In [ ]:
response.status_code

In [ ]:
len(response.text)

In [ ]:
topic_doc = BeautifulSoup(response.text,'html.parser')

In [ ]:
h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = topic_doc.find_all('h3',{'class':h3_selection_class })

repo_tags

In [ ]:
repo_tags[0].text.strip()

In [ ]:
a_tags = repo_tags[0].find_all('a')

In [ ]:
a_tags[0].text.strip()

In [ ]:
a_tags[1].text.strip()

In [ ]:
base_urls = 'https://github.com'
repo_url = base_url + a_tags[1]['href']
print(repo_url)

In [ ]:
star_tags = topic_doc.find_all('span',{'class': 'Counter js-social-count'})

In [ ]:
len(star_tags)

In [ ]:
star_tags[0].text.strip()

In [ ]:
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1])*1000)
    return int(stars_str)

In [ ]:
parse_star_count(star_tags[0].text.strip())

In [ ]:
def get_repo_info(h3_tag, star_tag):
    #returns all the required information about a repository
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars =  parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

In [ ]:
get_repo_info(repo_tags[0], star_tags[0])

In [ ]:
topics_repos_dict = {
    'UserName': [],
    'Repo_Name': [],
    'Stars': [],
    'Repo_URL': []
}

for i in range(len(repo_tags)) :
    repo_info = get_repo_info(repo_tags[i], star_tags[i])
    topics_repos_dict['UserName'].append(repo_info[0])
    topics_repos_dict['Repo_Name'].append(repo_info[1])
    topics_repos_dict['Stars'].append(repo_info[2])
    topics_repos_dict['Repo_URL'].append(repo_info[3])

In [ ]:
import os

def get_topic_page(topic_url):
    
    # download the page
    response = requests.get(topic_url)
    
   # check successfull response
    if response.status_code != 200:
          raise Exception ('Failed to load page{}'.format(topic_url))
            
   # parse using beautifulsoup
    topic_doc = BeautifulSoup(response.text, 'html.parser')   
    return topic_doc

def get_repo_info(h3_tag, star_tag):
    
    #returns all the required information about a repository
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars =  parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

def get_topic_repos(topic_doc) :
  
   # get the h3 tags containing repo title,repo url and username
    h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3',{'class':h3_selection_class })
    
   #  get star tags 
    star_tags = topic_doc.find_all('span',{'class': 'Counter js-social-count'})
    
   # create dict
    topic_repos_dict = {
                'UserName': [],
                'Repo_Name': [],
                'Stars': [],
                'Repo_URL': []}
    
   # get repo_info
    for i in range(len(repo_tags)) :
                repo_info = get_repo_info(repo_tags[i], star_tags[i])
                topic_repos_dict['UserName'].append(repo_info[0])
                topic_repos_dict['Repo_Name'].append(repo_info[1])
                topic_repos_dict['Stars'].append(repo_info[2])
                topic_repos_dict['Repo_URL'].append(repo_info[3])
                
    return pd.DataFrame(topic_repos_dict) 

def scrape_topic(topic_url, topic_name):
    
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(topic_name + 'csv', index = None)

In [ ]:
url4 = topic_urls[4]

In [ ]:
url4

In [ ]:
topic4_doc = get_topic_page(url4)

In [ ]:
topic4_repos = get_topic_repos(topic4_doc)

In [ ]:
topic4_repos

In [ ]:
topic_urls[6]

In [ ]:
 get_topic_repos(get_topic_page(topic_urls[6])).to_csv('ansible.csv',index= None)

#### Write a single function to :
###### 1. Get the list of topics from the topics page
###### 2. Get the list of top repos from the individual topic pages
###### 3. For each topic , create a csv of the top repos for the topic

In [ ]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles

def get_topic_descriptions(doc):
    selection_class = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': selection_class})
    topic_descriptions = []
    for tag in topic_desc_tags:
         topic_descriptions.append(tag.text.strip())
    return topic_descriptions  

def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-grow-0'})
    topic_urls = []
    base_url = 'https://github.com'
    
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag['href'])
    return topic_urls
    
    
def scrape_topics():
    topic_urls = 'https://github.com/topics'
    response = requests.get(topic_urls)   
    if response.status_code != 200:
        raise Exception ('Failed to load page{}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    topics_dict = {
        'Title' : get_topic_titles(doc),
        'Description' : get_topic_descriptions(doc),
        'URL' : get_topic_urls(doc)
    }   
    return pd.DataFrame(topics_dict)
    



In [ ]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topic_df = scrape_topics()
    for index, row in topic_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['Title']))
        scrape_topic(row['URL'],row['Title'])
    
    

In [ ]:
scrape_topics_repos()